<a href="https://colab.research.google.com/github/buzo14/SQL-DannyMa-Challenge-01/blob/main/Danny's_Dinner_SQL_challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement

Danny wants to use data to answer a few simple questions about his customers, especially about their visiting patterns, how much money they have spent and also which menu items are their favourite. Having this deeper connection with his customers will help him deliver a better and more personalised experience for his loyal customers.

He plans on using these insights to help him decide whether he should expand the existing customer loyalty program - additionally he needs help to generate some basic datasets so his team can easily inspect the data without needing to use SQL.

Danny has provided you with a sample of his overall customer data due to privacy issues - but he hopes that these examples are enough for you to write fully functioning SQL queries to help him answer his questions!

Danny has shared with you 3 key datasets for this case study:

- Sales
- Menu
- Members

![](DannyMa01.PNG)



In [1]:
import pandas as pd
import sqlite3

We import the necessary libraries, then create a connection to an empty database - danny_diner. We use the conn.cursor() syntax to enable us run SQL commands.

In [2]:
try:
    conn = sqlite3.connect('danny_diner.db') # Creates connection to existing db or creates an empty db
    print('Connected successfully...')
except Exception as e:
    print('Error during connection:', str(e))
else:
    c = conn.cursor()

Connected successfully...


Next up, we create our tables.

In [3]:
c.execute( '''
        CREATE TABLE sales (
          "customer_id" VARCHAR(1),
          "order_date" DATE,
          "product_id" INTEGER
        ); ''')

c.execute('''
        INSERT INTO sales
          ("customer_id", "order_date", "product_id")
        VALUES
          ('A', '2021-01-01', '1'),
          ('A', '2021-01-01', '2'),
          ('A', '2021-01-07', '2'),
          ('A', '2021-01-10', '3'),
          ('A', '2021-01-11', '3'),
          ('A', '2021-01-11', '3'),
          ('B', '2021-01-01', '2'),
          ('B', '2021-01-02', '2'),
          ('B', '2021-01-04', '1'),
          ('B', '2021-01-11', '1'),
          ('B', '2021-01-16', '3'),
          ('B', '2021-02-01', '3'),
          ('C', '2021-01-01', '3'),
          ('C', '2021-01-01', '3'),
          ('C', '2021-01-07', '3'); ''')


In [4]:
c.execute('''
        CREATE TABLE menu (
          "product_id" INTEGER,
          "product_name" VARCHAR(5),
          "price" INTEGER
        );''')

c.execute('''
        INSERT INTO menu
          ("product_id", "product_name", "price")
        VALUES
          ('1', 'sushi', '10'),
          ('2', 'curry', '15'),
          ('3', 'ramen', '12'); ''')


In [5]:
c.execute('''
        CREATE TABLE members (
          "customer_id" VARCHAR(1),
          "join_date" DATE
        );''')

c.execute('''
        INSERT INTO members
          ("customer_id", "join_date")
        VALUES
          ('A', '2021-01-07'),
          ('B', '2021-01-09'); ''')

conn.commit()

Using pd.read_sql, we read our tables into dataframes.

In [ ]:
sales_df = pd.read_sql('''
SELECT * FROM sales''', conn)


In [7]:
menu_df = pd.read_sql('''
SELECT * FROM menu''', conn)


In [8]:
members_df = pd.read_sql('''
SELECT * FROM members''', conn)


### Case Study Questions

We would be asking questions and finding solutions to help danny understand his business and customers better.


### 1. What is the total amount each customer spent at the restaurant?

In [12]:
customer_total = pd.read_sql(
  '''SELECT s.customer_id, SUM(m.price) AS total_amount_spent
      FROM sales AS s
     INNER JOIN menu m
     USING(product_id)
GROUP BY s.customer_id
ORDER BY total_amount_spent DESC;''', conn)

customer_total

,customer_id,total_amount_spent
0,A,76
1,B,74
2,C,36


### 2. How many days has each customer visited the restaurant?

In [14]:
customer_visits = pd.read_sql('''
          SELECT customer_id,COUNT(DISTINCT order_date) AS number_of_days
                  FROM sales
                  GROUP BY customer_id
                  ORDER BY number_of_days DESC; ''', conn)

customer_visits

,customer_id,number_of_days
0,B,6
1,A,4
2,C,2


### 3. What was the first item from the menu purchased by each customer?

In [16]:
first_purchase = pd.read_sql('''

WITH first_purchase AS(SELECT customer_id, MIN(order_date) AS min_date
	    FROM sales
      GROUP BY customer_id)

SELECT DISTINCT s.customer_id, f.min_date, s.product_id, m.product_name
	    FROM sales AS s
	 	  INNER JOIN first_purchase AS f
		  ON f.customer_id = s.customer_id AND f.min_date = s.order_date
	 	  INNER JOIN menu m
		  ON m.product_id = s.product_id
	    ORDER BY s.customer_id;

''', conn)

first_purchase

,customer_id,min_date,product_id,product_name
0,A,2021-01-01,1,sushi
1,A,2021-01-01,2,curry
2,B,2021-01-01,2,curry
3,C,2021-01-01,3,ramen


### 4. What is the most purchased item on the menu and how many times was it purchased by all customers?

In [20]:
most_purchased = pd.read_sql('''
  SELECT m.product_id, m.product_name, COUNT(*) AS number_of_orders
	      FROM sales AS s
	      INNER JOIN menu m
        USING(product_id)
	      GROUP BY m.product_id, m.product_name
	      ORDER BY number_of_orders DESC
	      LIMIT 1;
''', conn)

most_purchased

,product_id,product_name,number_of_orders
0,3,ramen,8


### 5. Which item was the most popular for each customer?

In [21]:
popular_items = pd.read_sql('''
  SELECT customer_id, product_name
	      FROM(SELECT s.customer_id, m.product_name, COUNT(*) AS number_of_orders, RANK() OVER(PARTITION BY customer_id ORDER BY COUNT(*) DESC) AS row_num
	FROM sales AS s
		 INNER JOIN menu m
		 USING(product_id)
	GROUP BY s.customer_id, m.product_name) AS popular
	WHERE row_num = 1
	ORDER BY customer_id;
''', conn)

popular_items

,customer_id,product_name
0,A,ramen
1,B,sushi
2,B,ramen
3,B,curry
4,C,ramen


### 6. Which item was purchased first by the customer after they became a member?

In [19]:
customer_first_purchase = pd.read_sql('''
    SELECT customer_id, product_name
	        FROM(SELECT s.customer_id, s.order_date, mem.join_date, s.product_id, m.product_name, ROW_NUMBER() OVER(PARTITION BY s.customer_id ORDER BY s.order_date) AS row_num
			        FROM sales AS s
		 	        INNER JOIN members mem
		 	        ON mem.customer_id = s.customer_id AND mem.join_date < s.order_date
		 	        INNER JOIN menu m
		 	        ON m.product_id = s.product_id) AS first_purchase
	            WHERE row_num = 1
	            ORDER BY customer_id;
''', conn)

customer_first_purchase

,customer_id,product_name
0,A,ramen
1,B,sushi


### 7. Which item was purchased just before the customer became a member?

In [22]:
new_member_purchase = pd.read_sql('''
    SELECT customer_id, product_name
	          FROM(SELECT s.customer_id, s.order_date, mem.join_date, s.product_id, m.product_name, RANK() OVER(PARTITION BY s.customer_id ORDER BY s.order_date DESC) AS row_num
			          FROM sales AS s
		 	          INNER JOIN members mem
		 	          ON mem.customer_id = s.customer_id AND mem.join_date > s.order_date
		 	          INNER JOIN menu m
		          	ON m.product_id = s.product_id) AS last_purchase
	              WHERE row_num = 1
	              ORDER BY customer_id;
''', conn)

new_member_purchase

,customer_id,product_name
0,A,sushi
1,A,curry
2,B,sushi


### 8. What is the total items and amount spent by each member before they became a member?

In [24]:
walkin_customer_spend = pd.read_sql('''

    WITH former_sales AS(
	            SELECT s.customer_id, s.order_date, m.join_date, s.product_id, me.product_name, me.price
		              FROM sales AS s
		              INNER JOIN members m
		              ON m.customer_id = s.customer_id AND m.join_date > s.order_date
		              INNER JOIN menu me
		              ON me.product_id = s.product_id)

      SELECT customer_id, COUNT(*) AS number_of_orders, SUM(price) AS amount_spent
        FROM former_sales
        GROUP BY customer_id
        ORDER BY customer_id;
''', conn)

walkin_customer_spend

,customer_id,number_of_orders,amount_spent
0,A,2,25
1,B,3,40


### 9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier how many points would each customer have?

In [25]:
loyalty_calculation = pd.read_sql('''
        SELECT s.customer_id, SUM(CASE WHEN m.product_name = 'sushi' THEN 10 * 2 * m.price
                      ELSE 10 * m.price END) AS cumulative_points
                      FROM sales AS s
                      INNER JOIN menu AS m
                        ON s.product_id = m.product_id
                      GROUP BY s.customer_id
                      ORDER BY customer_id;
''', conn)

loyalty_calculation

,customer_id,cumulative_points
0,A,860
1,B,940
2,C,360


### 10. Within the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [32]:
members_loyalty_calc = pd.read_sql('''
    WITH cust_points AS(
        SELECT s.customer_id
            , s.order_date
            , mm.join_date
            , DATE(mm.join_date, '+6 day') AS end_promo
            , s.product_id
            , m.price
            , CASE
                WHEN s.product_id = 1
                    THEN m.price * 20
                WHEN s.product_id != 1 AND
                (s.order_date BETWEEN mm.join_date AND DATE(mm.join_date,'+6 day'))
                    THEN (m.price * 20)
                ELSE m.price * 10
                END AS points
        FROM sales s
        JOIN members mm USING(customer_id)
        JOIN menu m USING(product_id)
        WHERE
            s.order_date <= '2021-01-31')

SELECT customer_id, SUM(points)  AS total
        FROM cust_points
        GROUP BY customer_id;
        ''', conn)

members_loyalty_calc

,customer_id,total
0,A,1370
1,B,820


In [33]:
conn.close()